In [1]:
# GAN : 적대적 생성 신경망 + CNN   DCGAN
# 생성모델(G)  식별모델 D
# G : k차원의 잠재적 특이 벡터를 입력값으로 받아서 대상과 동일형식의 데이터를 생셩하는 신경망
# D : 대상데이터를 입력값으로 받아 진위를식별하는 신경망

In [2]:
# GAN
# 1. 잠재적 특이 벡터 z를 난수로  생성하고 G(z)를 사용해 가짜 데이터(fake data)를 생성  fake_data <-- G(z)
# 2. fake_data를 D로식별한다  fake_data<-- D(real_data)
# 3. 진짜 데이터의 샘플(real_data)을 D로  식별한다  real_out<--- D(real_data)
# 4. fake_out 이 진짜 데이터라고 간주하고 크로스 엔트로피함수를 계산해서 G의 파라메터를  갱신
# 5. real_out이 진짜 데이터이고 fake_out이 가짜 데이터라고 간주하고 크로스 엔트로피 함수를 계산해서 D의 파라메터를 갱신

In [ ]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar -zxvf 102flowers.tgz

In [2]:
!mkdir oxford-102
!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg

In [3]:
!rmdir jpg

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder

In [14]:
img_data = ImageFolder('/content/oxford-102',
                       transform=transforms.Compose([
                           transforms.Resize(80),
                           transforms.CenterCrop(64),
                           transforms.ToTensor()
                       ])
                       )
batch_size = 64
img_loader = DataLoader(img_data, batch_size=batch_size, shuffle=True)

In [12]:
# data,label = next(iter(img_loader))

In [13]:
# data.shape, label

(torch.Size([64, 3, 64, 64]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [27]:
# 잠재적 특이 벡터 z를 100차원으로 만든, z  3 x 64 x 64 의 이미지를 만드는 생성모델을 구축
# ConvTransposed2d
nz = 100
ngf = 32
class GNet(nn.Module):
  def __init__(self):
    super(GNet, self).__init__()
    self.main = nn.Sequential(
        nn.ConvTranspose2d(nz, ngf*8, 4,1,0,bias=False  ), # 출력값에 편항을 추가하지 않아서 더 간단한 모델을 구현
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(inplace=True), # 출력값을 새로운 변수에 할당하지 않고 기존의 변수에 덮어쓴다.  메모리 사용량을 줄인다
        
        nn.ConvTranspose2d(ngf*8, ngf*4, 4,2,1,bias=False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(inplace=True),

        nn.ConvTranspose2d(ngf*4, ngf*2, 4,2,1,bias=False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(inplace=True),

        nn.ConvTranspose2d(ngf*2, ngf, 4,2,1,bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(inplace=True),

        nn.ConvTranspose2d(ngf, 3, 4,2,1,bias=False),
        nn.Tanh()
    )
  def forward(self, x):
    return self.main(x)

In [24]:
# 식별모델은 3 x 64 x 64 이미지를 최종적으로 1차원 스칼라로 변환하는 신경망
# 원래 논문에서 선형 계층을 사용하지 않음

ndf = 32
class DNet(nn.Module):
  def __init__(self) -> None:
    super(DNet,self).__init__()
    self.main = nn.Sequential(
        nn.Conv2d(3,ndf, 4,2,1,bias=False),
        nn.LeakyReLU(0.2,inplace=True),       

        nn.Conv2d(ndf,ndf*2, 4,2,1,bias=False),
        nn.BatchNorm2d(ndf*2),
        nn.LeakyReLU(0.2,inplace=True),

        nn.Conv2d(ndf*2,ndf*4, 4,2,1,bias=False),
        nn.BatchNorm2d(ndf*4),
        nn.LeakyReLU(0.2,inplace=True),

        nn.Conv2d(ndf*4,ndf*8, 4,2,1,bias=False),
        nn.BatchNorm2d(ndf*8),
        nn.LeakyReLU(0.2,inplace=True),

        nn.Conv2d(ndf*8,1, 4,2,1,bias=False)
    )
  def forward(self, x):
    return self.main(x).squeeze()

In [25]:
# 5번 합성곱 연산을해서 3 x 64 x64 이미지가 최종적으로 1 x 1x 1이 된다. 
# A x 1 x B x1 불필요한 1이 들어있는 shape를 A x B 처럼 조정
# Conv2d 는 입력과 출력 모두(batch_size, channel, height, width)  -- (batch_size,1,1,1)--> 불필요한 차원을 삭제

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
d = DNet().to(device)
g = GNet().to(device)

In [29]:
#DCGAN 원 논문에 기술되어 있는 파라메터 값
out_d = optim.Adam(d.parameters(), lr = 0.0002,betas=(0.5,0.999))
out_g = optim.Adam(g.parameters(), lr = 0.0002,betas=(0.5,0.999))

# 크로스 엔트로피를 계산하기 위한 보조 변수
ones = torch.ones(batch_size).to(device)
zeros = torch.zeros(batch_size).to(device)
loss_f = nn.BCEWithLogitsLoss()  # 출력값을 logit으로 변환하여 시그모이들 함수를 적용한다음 binary cross entropy손실을 계산
# logit : 0.5를 임계치로 삼아서. 양성 음성 클래스 변환
# 로짓함수의 수식은 : logit(p) = log(p / (1 - p))   p는 0과 1사이의 확률

# 모니터링용 Z
fixed_z = torch.randn(batch_size, nz,1,1).to(device)